In [0]:
import scandir
import os
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
import rpy2.robjects as ro
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import dill
import random
import cyvcf
from hdfstorehelper import HDFStoreHelper
import statsmodels.api as sm
import statsmodels.formula.api as smf
import operator
import traceback
%load_ext rpy2.ipython
from rpy2.robjects import pandas2ri as p2r
p2r.activate()
r = ro.r
from collections import defaultdict

In [0]:
analysis_dir_notimp = "/home/cfriedline/eckertlab/gypsy_indiv/masked/analysis/samtools1.2_no_otis/notimputed/"
analysis_dir_imp = "/home/cfriedline/eckertlab/gypsy_indiv/masked/analysis/samtools1.2_no_otis/beagle40/"
hdf_notimp = HDFStoreHelper(os.path.join(analysis_dir_notimp, "isect.hd5"))
hdf_imp = HDFStoreHelper(os.path.join(analysis_dir_imp, "isect.hd5"))

In [0]:
analysis_dir = [analysis_dir_notimp, analysis_dir_imp]

In [0]:
hdfs = [hdf_notimp, hdf_imp]

In [0]:
pop_allele_freqs = []
for d in analysis_dir:
    infile = os.path.join(d, "pop_allele_freqs.dill")
    paf = dill.load(open(infile))
    pop_allele_freqs.append(paf)

In [0]:
snpsfile_df = []
for i, paf in enumerate(pop_allele_freqs):
    pops = sorted(paf)
    paf_data = defaultdict(defaultdict)
    for popn in pops:
        af = paf[popn]
        for snp in af:
            paf_data["%s_1" % snp][popn] = af.ix["P",snp]
            paf_data["%s_2" % snp][popn] = af.ix["Q",snp]        
    snpsfile_df.append(pd.DataFrame(paf_data).T)

In [0]:
for i,snps in enumerate(snpsfile_df):
    outfile = os.path.join(analysis_dir[i], "snpsfile")
    snps.to_csv(outfile, sep="\t", header=False, index=False)

```bash
~/g/src/bayenv2_public/bayenv2 -i snpsfile -p 6 -k 1000000 -r 187564
```